## Transfer_learning with Tensorflow Part 2 Fine Tunning

In [ ]:
# Check if we are using GPU
!nvidia-smi

Mon Jan  2 14:02:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions
In previous notebooks we have created a bunch of helper functions in previous notebooks

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-01-02 14:06:45--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-02 14:06:45 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import helper functions we are oin to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves,unzip_data, walk_through_dir

We have imported these helper functions from mrdbourke github

## Lets get data ready 

We will import the models using tf.keras.applications and apply them to our own problem recognizing images of food 

In [ ]:
# Get 10% of training data of 10 classes of Food101
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2023-01-02 14:15:14--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.240, 172.217.164.144, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   199MB/s    in 0.8s    

2023-01-02 14:15:14 (199 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [ ]:
# Check out how many images and subdirectories are there 
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test

In [ ]:
# Create training and test directories path
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [ ]:
import tensorflow as tf
IMG_SIZE =(224,224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                image_size = IMG_SIZE,
                                                                label_mode = "categorical",
                                                                batch_size = BATCH_SIZE)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [ ]:
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [ ]:
# Check out class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [ ]:
# See example of batch data
for images,labels in train_data_10_percent.take(1):
  print(images,labels)

tf.Tensor(
[[[[4.83571434e+01 5.23571434e+01 5.33571434e+01]
   [4.83571434e+01 5.23571434e+01 5.33571434e+01]
   [4.77142868e+01 5.12857132e+01 5.02857132e+01]
   ...
   [7.19234314e+01 6.99234314e+01 7.09234314e+01]
   [6.41529312e+01 6.21529312e+01 6.31529312e+01]
   [3.02340965e+01 2.82340965e+01 2.92340965e+01]]

  [[5.00714302e+01 5.40714302e+01 5.50714302e+01]
   [5.21377563e+01 5.61377563e+01 5.71377563e+01]
   [5.20153046e+01 5.55867348e+01 5.61581612e+01]
   ...
   [6.67448959e+01 6.47448959e+01 6.57448959e+01]
   [5.05915680e+01 4.85915680e+01 4.95915680e+01]
   [1.71938133e+01 1.51938124e+01 1.61938133e+01]]

  [[5.22908173e+01 5.62908173e+01 5.72908173e+01]
   [5.56428566e+01 5.96428566e+01 6.06428566e+01]
   [5.73571434e+01 6.09285698e+01 6.15000000e+01]
   ...
   [5.89386292e+01 5.69386292e+01 5.79386292e+01]
   [2.59896774e+01 2.39896774e+01 2.49896774e+01]
   [1.38672924e+01 1.18672924e+01 1.28672924e+01]]

  ...

  [[1.58081650e+02 1.64867355e+02 1.57071396e+02]
   [1

## Model 0 : Building a transfer learning model using the keras functional api

The functional api gives us more flexibility with oour models

In [ ]:
#  1. Creating base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model
base_model.trainable = False

# 3 Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224,224,3),name = "input_layer")

# 4. If using ResNet50V2 you will need to normalize inputs
#x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5.  Pass the inputs to the base models
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information,reduse number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# Create output activation layer
outputs = tf.keras.layers.Dense(10,activation="softmax",name="output_layers")(x)

# 8 combine the inputs with outputs into the model
model_0 = tf.keras.Model(inputs,outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# 10. Fit the model (we use less steps for validation so it's faster)
history_10_percent = model_0.fit(train_data_10_percent,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_10_percent),
                                 validation_data=test_data,
                                 # Go through less of the validation data so epochs are faster (we want faster experiments!)
                                 validation_steps=int(0.25 * len(test_data)), 
                                 # Track our model's training logs for visualization later
                                 callbacks=[create_tensorboard_callback(dir_name="transfer_learning",experiment_name= "10_percent_feature_extract")])
     

Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extract/20230102-150259
Epoch 1/5
24/24 [==============================] - 19s 271ms/step - loss: 1.8430 - accuracy: 0.4373 - val_loss: 1.3069 - val_accuracy: 0.7122
Epoch 2/5
24/24 [==============================] - 4s 154ms/step - loss: 1.0750 - accuracy: 0.7720 - val_loss: 0.8852 - val_accuracy: 0.8207
Epoch 3/5
24/24 [==============================] - 5s 187ms/step - loss: 0.7776 - accuracy: 0.8307 - val_loss: 0.7271 - val_accuracy: 0.8503
Epoch 4/5
24/24 [==============================] - 5s 185ms/step - loss: 0.6361 - accuracy: 0.8493 - val_loss: 0.6126 - val_accuracy: 0.8635
Epoch 5/5
24/24 [==============================] - 5s 187ms/step - loss: 0.5481 - accuracy: 0.8733 - val_loss: 0.5528 - val_accuracy: 0.8799
